# Hyperparameters

In [1]:
hparam = {
    'epoch': 10,
    'hid_dim': 15,
    'gru_layers': 2,
    'lr': 0.0001,
    'report': 1,
    'zero_h0': False,
    'train': True
}

# Configure Dataset

In [2]:
from pathlib import Path
from random import shuffle
import pandas as pd
import pickle

# Load tensor file(pickle) list
dpath = Path('./Dataset/matches_tensor')
mlist = Path('./Dataset/match_list.pkl')
if not mlist.exists():
    mat_lists = shuffle(list(dpath.glob('*')))
    with mlist.open('wb') as file: pickle.dump(mat_lists, mlist)
else:
    with mlist.open('rb') as file: mat_lists = pickle.load(file)

tr_set = mat_lists[:190000]
val_set = mat_lists[190000:200000]
te_set = mat_lists[200000:]

# Load match results as dictionaris
mat_results = pd.read_csv('./Dataset/results.csv', index_col=0).to_dict('index')

# Model

In [3]:
from model import BCEModel

M = BCEModel(hparam['hid_dim'], hparam['gru_layers'], hparam['lr'], hparam['zero_h0'])
param = 'normal_bce.pkl' if not hparam['zero_h0'] else 'zero_h0_bce.pkl'

# Train

In [4]:
if hparam['train']:
    for epoch in range(hparam['epoch']):
        for idx, tr_dat in enumerate(tr_set):
            # print training progress
            print(f'[Epoch {epoch+1:3d}] ({idx+1:6d}/{len(tr_set)}) processing {tr_dat[:-4]:<30s}', end='\r')

            with open(f'{dpath}/{tr_dat}', 'rb') as feat_file:
                features = pickle.load(feat_file)
            winner = mat_results[tr_dat[:-4]]['win']

            # train
            M.train(features, winner)

        # validation process
        if (epoch+1)%hparam['report'] == 0: # execute validation in {hparam['report']} epoch interval
            M.reset_result()
            for idx, val_dat in enumerate(val_set):
                # print validation progress
                print(f'[Validation for epoch {epoch+1:3d}] ({idx+1:6d}/{len(val_set)}) processing {val_dat[:-4]:<30s}', end='\r') 

                with open(f'{dpath}/{val_dat}', 'rb') as feat_file:
                    features = pickle.load(feat_file)
                winner = mat_results[val_dat[:-4]]['win']
                
                # validate
                M.validate(features, winner)

            acc, pre, rec, f1, loss = M.get_result()
            print(f'[Validation for epoch {epoch+1:3d}] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')
            M.save_parameters(param)

[Validation for epoch   1] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   2] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   3] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   4] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   5] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   6] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   7] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   8] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
[Validation for epoch   9] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000

# Test

In [5]:
from torch import cuda

M.load_parameters(param)
M.reset_result()
for idx, te_dat in enumerate(te_set):
    cuda.empty_cache()
    if not hparam['zero_h0']: save_path = './experiment/matches_scores/normal_bce/'
    else                    : save_path = './experiment/matches_scores/zero_h0_bce/'

    # print training progresss
    print(f'[Testing] ({idx+1:6d}/{len(te_set)}) processing {te_dat[:-4]:<30s}', end='\r')

    with open(f'{dpath}/{te_dat}', 'rb') as feat_file:
        features = pickle.load(feat_file)
    winner = mat_results[te_dat[:-4]]['win']

    M.test(features, winner, save_path, te_dat[:-4])

acc, pre, rec, f1, loss = M.get_result()
print(f'[Test result] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')

[Test result] Accuracy: 1.0000 / Precision: 1.0000 / Recall: 1.0000 / F1 score: 1.0000 / loss: 0.0000
